# SISU Digital Humanities: Textual and Language Analysis on Social Media<br />
### Session 5: Word Embeddings
Created by Tom van Nuenen (tom.van_nuenen@kcl.ac.uk) <br />


# Word Embeddings

Today, we'll have a look at word embeddings using Gensim's `word2vec` and `doc2vec` methods. 

The goal of word vector embedding models is to learn dense, numerical vector representations for each term in a corpus vocabulary. If successful, the vectors for each term encode information about the meaning or concept the term represents, as well as the relationship between it and other terms in the vocabulary. Word vector models are  fully unsupervised: they learn all of these meanings and relationships without any advance knowledge.

After working through today's notebook, you'll be able to:

1. Use Gensim's word2vec method to create word vectors for a corpus;
2. Use these word vectors to reflect on implicit binaries and normativities in your data;
3. Visualize topic models using K-means clustering and t-SNE.

In [101]:
# General
from pprint import pprint
from collections import Counter
import os
import re
import logging
import string
import pickle
import numpy as np
import pandas as pd
import smart_open
import multiprocessing 
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

# Gensim
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases, Phraser

# NLTK
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer 
wordnet_lemmatizer = WordNetLemmatizer()

# Spacy
import spacy 

# Plotting
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

# Clustering 
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
from sklearn.manifold import TSNE

# Suppressing warnings
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tomvannuenen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tomvannuenen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tomvannuenen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Preprocessing

We'll start by cleaning up our data a bit. Let's load it up.

In [61]:
# load into df
df_com = pd.read_csv("data/icc3-comments.csv", lineterminator='\n')

In [62]:
df_com

,id,author_id,parent_id,step,week_number,step_number,text,timestamp,likes,first_reported_at,first_reported_reason,moderation_state,moderated
0,16363017,2b065715-ca93-45a1-b129-fb015822bdde,NaN,1.10,1,1,Welcome to this run of our course! Here at the...,2016-10-10 00:21:15 UTC,15,NaN,NaN,NaN,NaN
1,16363067,2b065715-ca93-45a1-b129-fb015822bdde,NaN,1.20,1,2,Our Educator and Mentor team is eager to know ...,2016-10-10 00:26:08 UTC,1,NaN,NaN,NaN,NaN
2,16363229,d0adfa81-69cd-4a49-92f3-4a8092983376,NaN,1.10,1,1,"Hello, Professor Kulich. Happy and excited to ...",2016-10-10 00:46:01 UTC,1,NaN,NaN,NaN,NaN
3,16363369,d0adfa81-69cd-4a49-92f3-4a8092983376,NaN,1.10,1,1,Hello everyone. Good morning.This is the first...,2016-10-10 01:01:07 UTC,0,NaN,NaN,NaN,NaN
4,16363370,19cb3801-93be-4268-9b62-b012517b85ba,NaN,1.50,1,5,Well I'm a Chinese so I presume the most impor...,2016-10-10 01:01:19 UTC,4,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16433,18305884,516aea02-8174-4fc6-a1ae-29e12a3de9c1,NaN,5.50,5,5,You don't even have to move abroad to experien...,2016-12-02 23:50:40 UTC,0,NaN,NaN,NaN,NaN
16434,18310548,df09fea0-03f3-4589-8129-5f8dfd4728b6,NaN,4.15,4,15,Ms Smith is focused on her tasl and wants some...,2016-12-03 10:01:44 UTC,0,NaN,NaN,NaN,NaN
16435,18311382,9f2392b4-ee0d-4054-ad1d-ede45cccbc00,NaN,5.13,5,13,This course has covered a lot of ground in a r...,2016-12-03 11:11:03 UTC,0,NaN,NaN,NaN,NaN
16436,18312814,6b740a2b-dfdb-44ae-a63d-f8a540dffad9,17796920.0,5.13,5,13,"@REDACTED I'm sorry, but the code of conduct a...",2016-12-03 13:04:33 UTC,0,NaN,NaN,NaN,NaN


In [63]:
# Get rid of empty values and reset index
df_com = df_com[~df_com['text'].isin(['[removed]', '[deleted]' ])].dropna(subset=['text']).reset_index(drop=True)

Let's create a small function that cleans up our text by removing all escape-tabs and escape-newlines, as well as all non symbol characters (except for the dot). It also normalizes spaces to a single character and removes leading and trailing spaces.

In [64]:
def clean_text(text):
    # Normalize tabs and remove newlines
    no_tabs = text.replace('\t', ' ').replace('\n', '');
    # Remove all characters except A-Z and a dot.
    alphas_only = re.sub("[^a-zA-Z\.]", " ", no_tabs);
    # Normalize spaces to 1
    multi_spaces = re.sub(" +", " ", alphas_only);
    # Strip trailing and leading spaces
    no_spaces = multi_spaces.strip();
    return no_spaces

We can now use the Pandas `.apply` method, allowing us to apply a function along an axis of the DataFrame. We'll use a lambda function: a small stand-in function that can take arguments, but only one expression. This is what a lambda looks like. Do you see how it works?

In [65]:
df_com["text_clean"] = df_com["text"].apply(lambda x: clean_text(x))

We now have an additional column in our DataFrame with cleaned up text.

In [66]:
df_com.head()

,id,author_id,parent_id,step,week_number,step_number,text,timestamp,likes,first_reported_at,first_reported_reason,moderation_state,moderated,text_clean
0,16363017,2b065715-ca93-45a1-b129-fb015822bdde,NaN,1.1,1,1,Welcome to this run of our course! Here at the...,2016-10-10 00:21:15 UTC,15,NaN,NaN,NaN,NaN,Welcome to this run of our course Here at the ...
1,16363067,2b065715-ca93-45a1-b129-fb015822bdde,NaN,1.2,1,2,Our Educator and Mentor team is eager to know ...,2016-10-10 00:26:08 UTC,1,NaN,NaN,NaN,NaN,Our Educator and Mentor team is eager to know ...
2,16363229,d0adfa81-69cd-4a49-92f3-4a8092983376,NaN,1.1,1,1,"Hello, Professor Kulich. Happy and excited to ...",2016-10-10 00:46:01 UTC,1,NaN,NaN,NaN,NaN,Hello Professor Kulich. Happy and excited to j...
3,16363369,d0adfa81-69cd-4a49-92f3-4a8092983376,NaN,1.1,1,1,Hello everyone. Good morning.This is the first...,2016-10-10 01:01:07 UTC,0,NaN,NaN,NaN,NaN,Hello everyone. Good morning.This is the first...
4,16363370,19cb3801-93be-4268-9b62-b012517b85ba,NaN,1.5,1,5,Well I'm a Chinese so I presume the most impor...,2016-10-10 01:01:19 UTC,4,NaN,NaN,NaN,NaN,Well I m a Chinese so I presume the most impor...


Let's turn it into a list.

In [67]:
text_li = df_com['text_clean'].tolist()

Next, we'll create a function that uses NLTK's `sent_tokenize()` method. This tokenizer splits our texts into sentences, which in turn are split into tokens. We'll also remove stopwords.

In [68]:
def sentence_tokenize(text):
    sentence_doc = sent_tokenize(text)
    sentences = [gensim.utils.simple_preprocess(str(doc), deacc=True) for doc in sentence_doc]  # deacc=True removes punctuations
    stop = set(stopwords.words('english') + ['’', '“', '”', 'nbsp', 'http'])
    no_stop = [[word for word in sentence if word not in stop] for sentence in sentences]
    return no_stop

In [69]:
com_sent_li = [sentence_tokenize(text) for text in text_li]

Note that we now have a list (of comments) of lists (sentences) of lists (tokens). Let's index the first token of the first sentence of the first comment:

In [73]:
com_sent_li[0][0][0]

'welcome'

We actually don't need the comment-level demarcation for the rest of our analysis. We can *flatten* our `com_sent_li` object to do so – this way, we create a list (of sentences) of lists (tokens).

In [74]:
sent_li = []
for sentence in com_sent_li:
    for tokens in sentence:
        sent_li.append(tokens)

Writing the same in a list comprehension looks like this, by the way:

In [75]:
sent_li = [tokens for sentence in com_sent_li for tokens in sentence]

Next, let's create a trigrams model using Gensim's `Phrases` and `Phraser` classes:

In [76]:
bigram = Phrases(sent_li, min_count=5, threshold=80)
trigram = Phrases(bigram[sent_li], threshold=80)  
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

And let's run that model over our list of lists.

In [77]:
trigrams = [trigram_mod[bigram_mod[sentence]] for sentence in sent_li]

## Word2Vec

Let's create our word embeddings model. Its input is a text corpus (split up in sentences) and its output is a set of "vectors" in N dimensions. It allows us to group the vectors of similar words together in vectorspace. We can then reduce the dimensionality to visualize the results in a way humans can understand (such as in a 2-dimensional space), or to perform linear algebra in order to find how words are related.

Word2vec is one example of a word embeddings model. It learns by taking words and their contexts (e.g. sentences) into account, and can then try to predict other words. Given enough data, usage and contexts, word2vec can make accurate guesses about a word’s meaning based on its appearances. Those guesses can be used to establish a word’s association with other words (e.g. “man” is to “boy” what “woman” is to “girl”), or cluster documents and classify them by topic.

### How many cores?
Word2Vec can work using independent threads doing simultaneous training. In general, you'll never want to use more workers than the number of CPU cores you have in your machine. So let's check out how many you have.



In [78]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

8

We now instantiate and train our Word2Vec model, using the parameters below.

In [102]:
num_features = 300        # Word vector dimensionality (how many features each word will be given)
min_word_count = 2        # Minimum word count to be taken into account
num_workers = cores       # Number of threads to run in parallel (equal to your amount of cores)
context = 6               # Context window size
downsampling = 0 #1e-2    # Downsample setting for frequent words
seed_n = 1                # Seed for the random number generator (to create reproducible results) 
sg_n = 1                  # Skip-gram = 1, CBOW = 0

model = Word2Vec(trigrams, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, seed=seed_n, sg=sg_n)

That was it! We can save this model in a Gensim object.

In [103]:
model.save("word2vec.vec")

In [104]:
model = Word2Vec.load("word2vec.vec")

How many terms are in our vocabulary?

In [105]:
# Note that Gensim v3 uses model.wv.vocab, while v4 uses model.vocab - please change accordingly if necessary
print('{:,} terms in the vocabulary.'.format(len(model.wv.vocab)))

13,779 terms in the vocabulary.


### Getting related terms

With the information in our word embeddings model, we can try to find similarities between words that interest us (i.e. words that have a similar vector). Let's create a function that retrieves related terms to some input.

In [81]:
def get_related_terms(token, topn=20):
    """
    look up the topn most similar terms to token and print them as a formatted list
    """

    for word, similarity in model.most_similar(positive=[token], topn=topn):
        print(word, round(similarity, 3))

In [85]:
get_related_terms(u'european')

african 0.882
eastern 0.874
asian 0.863
immigrants 0.861
islamic 0.859
north_south 0.858
latin_american 0.853
africans 0.845
dialects 0.842
western 0.842
indigenous 0.842
predominantly 0.842
irish 0.842
europeans 0.841
colonization 0.841
origins 0.839
population 0.838
middle_eastern 0.838
ethnically 0.836
australian 0.835


### Word algebra

Word algebra, also known as analogy completion, means doing math with words (like the famous example "king - man + woman = queen". The core idea is that once words are represented as numerical vectors, you can do math with them. The mathematical procedure works as follows:

1. Provide a set of words or phrases you want to add or subtract.
2. Look up the vectors that represent those terms in the word vector model.
3. Add and subtract those vectors to produce a new, combined vector.
4. Look up the most similar vector(s) to this new, combined vector via cosine similarity.
5. Return the word(s) associated with the similar vector(s).

Let's try it out. We'll create a function that does this for us.

In [88]:
def word_algebra(add=[], subtract=[], topn=10):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = model.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

In [89]:
word_algebra(add=['cultural','problem'])

issue
obvious
causes
sides
root
forth
lies
function
emphasise
exhibit


In [90]:
word_algebra(add=['cultural','problem'], subtract=['man'])

variations
styles
hc_lc
interactions
able_recognize
adjustment
processes
categories
encounters
communicative


## Word Vector Visualization with t-SNE

t-Distributed Stochastic Neighbor Embedding, or t-SNE, is a dimensionality reduction technique to assist with visualizing high-dimensional datasets. It attempts to map high-dimensional data onto a low two- or three-dimensional representation. It tries to keep the relative distances between points as closely as possible in both high-dimensional and low-dimensional space.

Scikit-learn provides a convenient implementation of the t-SNE algorithm with its `TSNE` class.

Our input for t-SNE will be the DataFrame of word vectors we created before. Let's first drop some stopwords, and
take only the 5,000 most frequent terms in the vocabulary for time's sake.

First, we need to create a DataFrame with the terms as the row labels, and the 100 dimensions of the word vector model as the columns.

In [92]:
# build a list of the terms, integer indices, and term counts from the word2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count) for term, voc in model.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab.sort(key = lambda x: x[2])  

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the vectors as data, and the terms as row labels
word_vectors = pd.DataFrame(model.wv.syn0norm[term_indices, :], index=ordered_terms)

word_vectors

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
pens,-0.071777,-0.012775,-0.065915,-0.115861,0.069841,0.050224,0.084207,0.090258,-0.044392,0.041476,...,0.040967,-0.060461,-0.040834,0.024077,-0.039086,0.004867,0.014293,-0.045933,-0.016928,-0.034440
snake,-0.050524,-0.007629,-0.036474,-0.089833,0.085683,0.053400,0.082193,0.068238,-0.060112,0.036512,...,0.042715,-0.061790,-0.020632,0.034443,-0.042395,0.003541,-0.000640,-0.044510,-0.017364,-0.037284
hindsight,-0.072400,-0.013743,-0.081995,-0.127475,0.019953,0.021737,0.065020,0.069140,-0.036209,0.013391,...,0.052135,-0.060243,-0.060657,0.060296,-0.080406,0.004948,0.021079,-0.043984,0.002858,-0.054514
competently,-0.044828,-0.003042,-0.077056,-0.132246,0.028013,0.052331,0.071501,0.066206,-0.037323,0.023777,...,0.041571,-0.044194,-0.056700,0.053672,-0.051024,0.006473,0.025016,-0.028347,0.002246,-0.041878
safecreative,-0.059065,-0.012718,-0.058940,-0.118076,0.064464,0.017560,0.077404,0.072513,-0.034379,0.024794,...,0.056786,-0.058259,-0.036119,0.036368,-0.056881,0.008136,0.008696,-0.051099,-0.004120,-0.067410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
think,-0.113001,0.004582,-0.038043,-0.096367,0.013499,0.113994,0.137279,0.080626,0.000652,-0.049199,...,0.055704,0.057805,-0.018887,0.070893,-0.121991,0.002611,0.027904,-0.061192,-0.032370,-0.095199
cultural,-0.042113,0.050266,-0.034401,-0.047939,0.023789,0.018629,-0.051416,-0.065410,-0.018043,-0.013228,...,0.044619,-0.026098,0.051893,0.074644,-0.050740,0.044840,-0.009082,-0.008333,0.075245,0.026154
different,0.020864,-0.036423,-0.117106,-0.138592,-0.013320,0.051274,0.061817,0.086441,-0.045858,-0.044461,...,0.007419,-0.110917,-0.056807,0.093585,-0.042660,0.021983,-0.012540,0.049146,0.032113,0.038967
culture,-0.089575,0.009948,-0.089878,-0.064490,-0.028623,0.001827,0.059268,0.014839,-0.035972,-0.011894,...,0.109846,-0.048446,-0.005470,-0.016890,-0.042515,-0.064860,0.020312,-0.102158,-0.001769,0.014197


In [93]:
tsne = TSNE()
tsne_vectors = tsne.fit_transform(word_vectors.values)

In [94]:
tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(word_vectors.index),
                            columns=['x_coord', 'y_coord'])

In [95]:
tsne_vectors.head()

,x_coord,y_coord
pens,-16.694654,-32.048573
snake,-37.612736,-40.024883
hindsight,17.202772,-3.773781
competently,32.182758,18.246811
safecreative,12.749897,-35.108528


In [45]:
tsne_vectors['word'] = tsne_vectors.index

In [46]:
output_notebook()

Loading BokehJS ...

In [47]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= ('pan, wheel_zoom, box_zoom, box_select, reset, reset'),
                   active_scroll='wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools(HoverTool(tooltips = '@word'))

# draw the words as circles on the plot
tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value('14pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# show the plot
show(tsne_plot);

/Users/tomvannuenen/opt/anaconda3/lib/python3.8/site-packages/bokeh/plotting/_tools.py:132: UserWarning: ResetTool(id='1027', ...) are being repeated
  warnings.warn("%s are being repeated" % ",".join(repeated_tools))


## K-means clustering
Another thing we can do is cluster the words using K-Means clustering. 

K-Means clustering aims to partition N observations into K clusters in which each observation belongs to the cluster with the nearest mean (called the "cluster centre"), which serves as a prototype of the cluster.

Since our words are all represented as vectors, applying K-Means is easy to do since the clustering algorithm will simply look at differences between vectors (and centers).

In [96]:
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++');
    idx = kmeans_clustering.fit_predict(word_vectors);
    
    return kmeans_clustering.cluster_centers_, idx;

In [97]:
Z = model.wv.syn0

In [98]:
centers, clusters = clustering_on_wordvecs(Z, 10);
centroid_map = dict(zip(model.wv.index2word, clusters));

Next, we get words in each cluster that are closest to the cluster center. To do this, we initialize a KDTree on the word vectors, and query it for the Top K words on each cluster center. Using the Index 2 word dictionary, we than correspond each word vector back to it’s original word representation and add them to a dataframe for easier printing.

In [51]:
def get_top_words(index2word, k, centers, wordvecs):
    tree = KDTree(wordvecs);
    # Use closest points for each cluster center to query closest 20 points to it
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers];
    closest_words_idxs = [x[1] for x in closest_points];
    # Query Word Index  for each position in the above array, and added to a Dictionary
    closest_words = {};
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]
    # Create DataFrame from dictionary
    df = pd.DataFrame(closest_words);
    df.index = df.index+1
    return df

Let’s get the top words and print the first 20 in each cluster:

In [52]:
top_words = get_top_words(model.wv.index2word, 5000, centers, Z);

In [53]:
top_words[:10]

,Cluster #0,Cluster #1,Cluster #2,Cluster #3,Cluster #4,Cluster #5,Cluster #6,Cluster #7,Cluster #8,Cluster #9
1,hardware,globalised_world,anticipate,kansas,tendencies,use_google_calendar,professor_kulich,buying,florida,negatives
2,ableist,creating_shared_meanings,break_ice,starter,universally,monthly,comments_replies,cashier,quebec,dialogic
3,fleeing,advantage,disturbing,centres,derived,scheduling,worthwhile,hurry,decided_move,overtime
4,bemused,behavioural,intentionally,donald,socioeconomic,paper_diary,exercises,bumping,zimbabwe,preferably
5,abound,hinder,withdrawn,examination,caste,mobile_app,informative,cash,oversea,depict
6,insults,motivations,sth,lima,transmitted,alert,thankyou,sights,zambia,generated
7,engages,generalize,choosing,brahmins,familial,organised,ernesto,cup_tea,bangladesh,threats
8,definetely,acknowledging,agreeing,fermented,inequality,electronic_calendar,alot,broke,uruguay,discusses
9,rap,seeks,ambiguous,grandad,beings,diaries,proceed,promised,kerala,presumptions
10,discriminatory,shared_meanings,investigate,sky,attribute,scheduled,survey,laughing,philippines,transit


## **Doc2Vec**

You might be interested in the relationship between comments or posts, instead of single words. You can do this using tf-idf, as you saw in week 2. But word embeddings offer another unsupervised approach to find similarities.

Let's read our data again. We’ll need to associate a tag/number with each document of the training corpus. In our case, the tag is simply a zero-based line number.



In [55]:
def read_text(inp):
    for i, line in enumerate(inp):
        tokens = gensim.utils.simple_preprocess(line)
        yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

comments = list(read_text(df_com['text_clean']))

Instantiate a Doc2Vec model with a vector size with 50 dimensions and iterating over the training corpus 40 times. We set the minimum word count to 2 in order to discard words with very few occurrences.

In [56]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

Build the vocabulary:

In [57]:
model.build_vocab(comments)

Train the model on the corpus:

In [58]:
model.train(comments, total_examples=model.corpus_count, epochs=model.epochs)

To assess our new model, we’ll first infer new vectors for each document of the training corpus, compare the inferred vectors with the training corpus, and then returning the rank of the document based on self-similarity. Basically, we’re pretending as if the training corpus is some new unseen data and then seeing how they compare with the trained model. The expectation is that we’ve likely overfit our model (i.e., all of the ranks will be less than 2) and so we should be able to find similar documents very easily. Additionally, we’ll keep track of the second ranks for a comparison of less similar documents.

In [59]:
ranks = []
second_ranks = []
for doc_id in range(len(comments)):
    inferred_vector = model.infer_vector(comments[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

KeyboardInterrupt: 

Let's find some similar comments!

In [ ]:
print('Comment ({}): «{}»\n'.format(doc_id, ' '.join(comments[doc_id].words)))
print(u'SIMILAR/DISSIMILAR COMMENTS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(comments[sims[index][0]].words)))

And let's automate that a bit

In [ ]:
# Pick a random document from the corpus and infer a vector from the model
import random
doc_id = random.randint(0, len(comments) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(comments[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(comments[sim_id[0]].words)))
